In [1]:
%matplotlib notebook
from bayes_opt import BayesianOptimization
import numpy as np
from mpl_toolkits.mplot3d import axes3d
import matplotlib.pyplot as plt
from matplotlib import gridspec
from matplotlib import cm
from scipy.optimize import minimize_scalar, LinearConstraint, minimize
from scipy.stats import norm
from sklearn.gaussian_process import GaussianProcessRegressor

In [2]:
def target(x1,x2):
    return  -1 * ((4 - (2.1 * (x1**2)) + (x1**4)/3) * (x1**2) + x1*x2 + (-4 + 4*(x2**2)) * x2**2)
def target1D(x):
    return  -1 * ((4 - (2.1 * (x[0]**2)) + (x[0]**4)/3) * (x[0]**2) + x[0]*x[1] + (-4 + 4*(x[1]**2)) * x[1]**2)

In [3]:
X1Vals = np.linspace(-3,3,100)
X2Vals = np.linspace(-2,2,100)
X1, X2 = np.meshgrid(X1Vals, X2Vals)
Y = target(X1,X2)

In [4]:
def learnLengthscale():
    return 1

In [20]:
def negativeExpectedImprovement(x, gp, dataset):
    # because we are maximizing -1 * 6C we want to solve max_for x in x(0, f(x) - f(x^))
    xi = 0.01
    x = x.reshape(1,-1)
    y_mean, y_std = gp.predict(x, return_std=True)
    currentBest = np.max(dataset['z'])
    if y_std > 0:
        Z = (y_mean - currentBest - xi) / y_std
        ei = (y_mean - currentBest - xi) * norm.cdf(Z) + y_std * norm.pdf(Z)
    else:
        ei = 0
    return  -1 * ei

def negativeProbabilityOfImprovement(x, gp, dataset):
    currentBest = np.max(dataset['z'])
    x = x.reshape(1,-1)
    y_mean, y_std = gp.predict(x, return_std=True)
    return -1 * (currentBest - y_mean) / y_std

def ABO(x0, y0, z0, budget=50, lowerTimeDelta=0.01, upperBoundThreshold=0.2, lengthScale=None, timeGranularity=0.1):
    # max at 1.0316
    dataset = {'x1': x0, 'x2':y0, 'z': z0}
    # defaults: GaussianProcessRegressor(kernel=None, alpha=1e-10, optimizer=’fmin_l_bfgs_b’, n_restarts_optimizer=0, normalize_y=False, copy_X_train=True, random_state=None)
    gp = GaussianProcessRegressor()
    for iteration in range(budget):
        # Train GP model
        gp.fit(np.stack([dataset['x1'], dataset['x2']], axis=1), dataset['z'])
        # Set current time from last collected sample
        tC = dataset["x1"][-1]
        #Set time lengthscale (currently implementing ABOF)
        if not lengthScale:
            # if lengthscale set as None, learn lengthscale
            lengthScale = learnLengthscale()
        # Set bounds 
        lb = tC + lowerTimeDelta
        ub = tC + upperBoundThreshold * lengthScale
        #bounds = ((tC + lowerTimeDelta, tC + upperBoundThreshold * lengthScale), (-2, 2))
        bounds = ((-3,3), (-2,2))
        #print("most recent time: {}".format(tC))
        #print("minimizing neg. exp. improv in range [{}, {}]".format(lb, ub))
        newX = 0
        newY = 0
        max_acq = 0
        for samplingIteration in range(25):
            x = np.random.uniform(low=-3,high=3)
            y = np.random.uniform(low=-2, high=2)
            #result = minimize(negativeExpectedImprovement,np.array([x,y]), args=(gp, dataset), bounds = bounds)
            result = minimize(negativeProbabilityOfImprovement, np.array([x,y]), args=(gp, dataset), bounds = bounds)
            if not result.success:
                print("ERROR: \n{}\n".format(result.message))
                continue
            # Store it if better than previous minimum(maximum).
            if max_acq is None or -result.fun >= max_acq:
                newX, newY = result.x
                max_acq = -result.fun
        newZ = target(newX, newY)
        dataset['x1'] = np.append(dataset['x1'], [newX])
        dataset['x2'] = np.append(dataset['x2'], [newY])
        dataset['z'] = np.append(dataset['z'], [newZ])
        print("Most recently sampled point: f({}, {}) = {}".format(dataset['x1'][-1], dataset['x2'][-1], dataset['z'][-1]))
        maxIDX = np.argmax(dataset['z'])
        print("f({},{}) = {}\n".format(dataset['x1'][maxIDX], dataset['x2'][maxIDX], dataset['z'][maxIDX]))
    return np.max(dataset['z'])

        

In [21]:
x0 = np.random.uniform(low=-3,high=3,size=5)
x1 = np.random.uniform(low=-2,high=2,size=5)
initialX0, initialX1 = np.meshgrid(x0,x1)
z = target(initialX0, initialX1)
print("seed max val: {}".format(np.max(z)))
ABO(initialX0.flatten(), initialX1.flatten(), z.flatten(), budget=25)

seed max val: 0.3439445536204493
ERROR: 
b'ABNORMAL_TERMINATION_IN_LNSRCH'

ERROR: 
b'ABNORMAL_TERMINATION_IN_LNSRCH'

ERROR: 
b'ABNORMAL_TERMINATION_IN_LNSRCH'

ERROR: 
b'ABNORMAL_TERMINATION_IN_LNSRCH'

ERROR: 
b'ABNORMAL_TERMINATION_IN_LNSRCH'

ERROR: 
b'ABNORMAL_TERMINATION_IN_LNSRCH'

ERROR: 
b'ABNORMAL_TERMINATION_IN_LNSRCH'

ERROR: 
b'ABNORMAL_TERMINATION_IN_LNSRCH'

ERROR: 
b'ABNORMAL_TERMINATION_IN_LNSRCH'

ERROR: 
b'ABNORMAL_TERMINATION_IN_LNSRCH'

ERROR: 
b'ABNORMAL_TERMINATION_IN_LNSRCH'

ERROR: 
b'ABNORMAL_TERMINATION_IN_LNSRCH'

ERROR: 
b'ABNORMAL_TERMINATION_IN_LNSRCH'

ERROR: 
b'ABNORMAL_TERMINATION_IN_LNSRCH'

ERROR: 
b'ABNORMAL_TERMINATION_IN_LNSRCH'

ERROR: 
b'ABNORMAL_TERMINATION_IN_LNSRCH'

ERROR: 
b'ABNORMAL_TERMINATION_IN_LNSRCH'

ERROR: 
b'ABNORMAL_TERMINATION_IN_LNSRCH'

Most recently sampled point: f(1.0571642123995542, 1.0597992322722363) = -3.9865078143252033
f(-0.5349402228389746,0.8172420755258041) = 0.3439445536204493

ERROR: 
b'ABNORMAL_TERMINATION_IN_LN

/Users/danielzeiberg/miniconda3/lib/python3.6/site-packages/sklearn/gaussian_process/gpr.py:335: UserWarning: Predicted variances smaller than 0. Setting those variances to 0.
  warnings.warn("Predicted variances smaller than 0. "
/Users/danielzeiberg/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:18: RuntimeWarning: divide by zero encountered in true_divide
/Users/danielzeiberg/miniconda3/lib/python3.6/site-packages/scipy/optimize/optimize.py:663: RuntimeWarning: invalid value encountered in subtract
  grad[k] = (f(*((xk + d,) + args)) - f0) / d[k]


ERROR: 
b'ABNORMAL_TERMINATION_IN_LNSRCH'

ERROR: 
b'ABNORMAL_TERMINATION_IN_LNSRCH'

ERROR: 
b'ABNORMAL_TERMINATION_IN_LNSRCH'

ERROR: 
b'ABNORMAL_TERMINATION_IN_LNSRCH'

Most recently sampled point: f(-1.1174689617916584, -0.6279369587076977) = -2.1157815271646965
f(-0.5349402228389746,0.8172420755258041) = 0.3439445536204493

ERROR: 
b'ABNORMAL_TERMINATION_IN_LNSRCH'

ERROR: 
b'ABNORMAL_TERMINATION_IN_LNSRCH'

ERROR: 
b'ABNORMAL_TERMINATION_IN_LNSRCH'

ERROR: 
b'ABNORMAL_TERMINATION_IN_LNSRCH'

ERROR: 
b'ABNORMAL_TERMINATION_IN_LNSRCH'

ERROR: 
b'ABNORMAL_TERMINATION_IN_LNSRCH'

ERROR: 
b'ABNORMAL_TERMINATION_IN_LNSRCH'

ERROR: 
b'ABNORMAL_TERMINATION_IN_LNSRCH'

ERROR: 
b'ABNORMAL_TERMINATION_IN_LNSRCH'

ERROR: 
b'ABNORMAL_TERMINATION_IN_LNSRCH'

ERROR: 
b'ABNORMAL_TERMINATION_IN_LNSRCH'

ERROR: 
b'ABNORMAL_TERMINATION_IN_LNSRCH'

ERROR: 
b'ABNORMAL_TERMINATION_IN_LNSRCH'

ERROR: 
b'ABNORMAL_TERMINATION_IN_LNSRCH'

ERROR: 
b'ABNORMAL_TERMINATION_IN_LNSRCH'

ERROR: 
b'ABNORMAL_TERMI

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').